# file loading

In [1]:
import requests
import zipfile
import io

# 구글 드라이브 공유 링크에서 파일 ID를 가져옵니다.
file_id = '1ZUueEhjvhzmo8UpZLwqMgzMKA6FGtLJk'  # 여기에 파일 ID를 입력하세요.
download_url = f'https://drive.google.com/uc?export=download&id={file_id}'

# 파일 다운로드
response = requests.get(download_url)
response.raise_for_status()  # 요청이 실패하면 예외를 발생시킵니다.

# 다운로드한 ZIP 파일을 메모리에서 직접 압축 해제합니다.
with zipfile.ZipFile(io.BytesIO(response.content)) as zip_ref:
    zip_ref.extractall('/content/unzipped_folder')

# 결과 확인
import os
os.listdir('/content/unzipped_folder')


['train.csv', 'test.csv', 'sample_submission.csv']

# data loading

In [2]:
import pandas as pd
import numpy as np

In [16]:
# 데이터 불러오기
df = pd.read_csv("/content/unzipped_folder/train.csv")
#train = pd.read_csv("/content/unzipped_folder/train.csv")
#test = pd.read_csv("/content/unzipped_folder/test.csv")

In [17]:
print(df.head())

           ID SUBCLASS    A2M AAAS AADAT AARS1 ABAT ABCA1 ABCA2 ABCA3  ...  \
0  TRAIN_0000    KIPAN     WT   WT    WT    WT   WT    WT    WT    WT  ...   
1  TRAIN_0001     SARC     WT   WT    WT    WT   WT    WT    WT    WT  ...   
2  TRAIN_0002     SKCM  R895R   WT    WT    WT   WT    WT    WT    WT  ...   
3  TRAIN_0003     KIRC     WT   WT    WT    WT   WT    WT    WT    WT  ...   
4  TRAIN_0004   GBMLGG     WT   WT    WT    WT   WT    WT    WT    WT  ...   

  ZNF292 ZNF365 ZNF639 ZNF707 ZNFX1 ZNRF4 ZPBP ZW10 ZWINT ZYX  
0     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
1     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
2     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
3     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  
4     WT     WT     WT     WT    WT    WT   WT   WT    WT  WT  

[5 rows x 4386 columns]


# preprocessing

In [18]:
# 정규 표현식 regular expression 모듈
import re

# 패턴을 구분하는 함수 정의
def process_value(value):
    # 1) WT 그대로 유지
    if value == 'WT':
        return value

    # 2) 복합 패턴 처리를 위한 split
    parts = value.split()  # 공백 기준
    processed_parts = []

    for part in parts:
        # 숫자 앞뒤로 같은 패턴이 있는지 확인
        match = re.match(r'([A-Za-z]+)(\d+)([A-Za-z]+)', part)
        if match:
            prefix, number, suffix = match.groups()
            # 앞뒤 문자가 같다면 해당 부분 무시
            if prefix == suffix:
                continue
        processed_parts.append(part)

    # 처리된 부분이 없으면 WT로 변경
    return 'WT' if not processed_parts else ' '.join(processed_parts)

# 데이터프레임 전처리
for col in df.columns[2:]:  # ID와 SUBCLASS 제외
    df[col] = df[col].apply(process_value)

# 최종 데이터 확인
print(df)


              ID SUBCLASS A2M AAAS AADAT AARS1 ABAT ABCA1 ABCA2 ABCA3  ...  \
0     TRAIN_0000    KIPAN  WT   WT    WT    WT   WT    WT    WT    WT  ...   
1     TRAIN_0001     SARC  WT   WT    WT    WT   WT    WT    WT    WT  ...   
2     TRAIN_0002     SKCM  WT   WT    WT    WT   WT    WT    WT    WT  ...   
3     TRAIN_0003     KIRC  WT   WT    WT    WT   WT    WT    WT    WT  ...   
4     TRAIN_0004   GBMLGG  WT   WT    WT    WT   WT    WT    WT    WT  ...   
...          ...      ...  ..  ...   ...   ...  ...   ...   ...   ...  ...   
6196  TRAIN_6196     LUAD  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6197  TRAIN_6197      LGG  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6198  TRAIN_6198     COAD  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6199  TRAIN_6199     TGCT  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6200  TRAIN_6200     SKCM  WT   WT    WT    WT   WT    WT    WT    WT  ...   

     ZNF292 ZNF365 ZNF639 ZNF707 ZNFX1 ZNRF4 ZPBP ZW10  ZWINT Z

In [19]:
# 변이 횟수를 계산하는 함수 정의
def count_mutations(row):
    mutation_count = 0
    for val in row:
        if val != 'WT':
            # 공백으로 나누어진 패턴 개수를 모두 더하기
            mutation_count += len(val.split())
    return mutation_count

# 각 ID별로 변이 횟수 측정하여 새로운 열 추가
df['Mutation_Count'] = df.iloc[:, 2:].apply(count_mutations, axis=1)

# 최종 데이터 확인
print(df)

              ID SUBCLASS A2M AAAS AADAT AARS1 ABAT ABCA1 ABCA2 ABCA3  ...  \
0     TRAIN_0000    KIPAN  WT   WT    WT    WT   WT    WT    WT    WT  ...   
1     TRAIN_0001     SARC  WT   WT    WT    WT   WT    WT    WT    WT  ...   
2     TRAIN_0002     SKCM  WT   WT    WT    WT   WT    WT    WT    WT  ...   
3     TRAIN_0003     KIRC  WT   WT    WT    WT   WT    WT    WT    WT  ...   
4     TRAIN_0004   GBMLGG  WT   WT    WT    WT   WT    WT    WT    WT  ...   
...          ...      ...  ..  ...   ...   ...  ...   ...   ...   ...  ...   
6196  TRAIN_6196     LUAD  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6197  TRAIN_6197      LGG  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6198  TRAIN_6198     COAD  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6199  TRAIN_6199     TGCT  WT   WT    WT    WT   WT    WT    WT    WT  ...   
6200  TRAIN_6200     SKCM  WT   WT    WT    WT   WT    WT    WT    WT  ...   

     ZNF365 ZNF639 ZNF707 ZNFX1 ZNRF4 ZPBP ZW10  ZWINT ZYX Muta

<ipython-input-19-95f213850116>:11: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Mutation_Count'] = df.iloc[:, 2:].apply(count_mutations, axis=1)
